In [1]:
# Imports

# Environment
from airlift.envs.airlift_env import AirliftEnv
from airlift.envs import PlaneType
from airlift.envs.generators.map_generators import PlainMapGenerator

# Generators
from airlift.envs.generators.world_generators import AirliftWorldGenerator
from airlift.envs.generators.airport_generators import RandomAirportGenerator
from airlift.envs.generators.route_generators import RouteByDistanceGenerator
from airlift.envs.generators.airplane_generators import AirplaneGenerator
from airlift.envs.generators.cargo_generators import StaticCargoGenerator

# Dynamic events
from airlift.envs.events.event_interval_generator import EventIntervalGenerator
from airlift.envs.generators.cargo_generators import DynamicCargoGenerator

# Solution
from solution.mysolution import MySolution

# Helper methods
from airlift.solutions import doepisode
from eval_solution import write_results
from solution.models.v2.model_v2 import Policy, Value, get_config
from solution.format_obs import format_obs
from solution.post_process import post_process
import torch
import torch.nn as nn
import random
from torch.distributions import Categorical
from tqdm import tqdm
import networkx as nx

In [2]:
# Scheduler

class Scheduler:
    def __init__(self, seed=0, num_episodes=1, ending_epsilon=0.2, starting_epsilon=1):
        self.episode = 1
        self.num_episodes = num_episodes
        self.ending_epsilon = ending_epsilon
        self.starting_epsilon = starting_epsilon
    

    def get_next_epsilon(self):
        """
        Returns epsilon for this episode
        Epsilon is scheduled quadratically, meaning the following
            would be the epsilon values for 10 episodes with defaults
             1: 0.992
             2: 0.968
             3: 0.928
             4: 0.872
             5: 0.800
             6: 0.712
             7: 0.608
             8: 0.488
             9: 0.352
            10: 0.200
        """
        return self.starting_epsilon - ((self.episode / self.num_episodes)**2)*\
            (self.starting_epsilon-self.ending_epsilon)


    def small_env_with_purturbations(self, max_cycles=5000):
        """
        Returns a preset environment
        - 8 nodes
        - 2 airplane
        - purturbations present
        """

        # Decide plane types
        plane_types = [PlaneType(id=0, max_range=1.0, speed=0.05, max_weight=5)]

        # Decide airport generator
        airport_generator = RandomAirportGenerator(
            max_airports=3,
            make_drop_off_area=True,
            make_pick_up_area=True,
            num_drop_off_airports=1,
            num_pick_up_airports=1,
            mapgen=PlainMapGenerator(),
        )

        # Decide route generator
        route_generator = RouteByDistanceGenerator(
            route_ratio=2,
            poisson_lambda=1/2,
            malfunction_generator=EventIntervalGenerator(
                                        min_duration=10,
                                        max_duration=30),
        )

        # Decide cargo generator
        cargo_generator = DynamicCargoGenerator(
            cargo_creation_rate=1/100,
            max_cargo_to_create=5,
            num_initial_tasks=10,
            max_weight=3,
            max_stagger_steps=max_cycles/2,
            soft_deadline_multiplier=10,
            hard_deadline_multiplier=20,
        )

        # Decide the number of agents
        num_agents = 2

        # Create the environment
        env = AirliftEnv(
                world_generator=AirliftWorldGenerator(
                plane_types=plane_types,
                airport_generator=airport_generator,
                route_generator=route_generator,
                cargo_generator=cargo_generator,
                airplane_generator=AirplaneGenerator(num_of_agents=num_agents),
                max_cycles=max_cycles
                ),
            )
        
        return env


    def hardest_env(self, max_cycles=5000):
        """
        Returns a preset environment
        - 8 nodes
        - 2 airplane
        - purturbations present
        """

        num_agents = 244
        plane_types = [
            PlaneType(id=0, max_range=1.0, speed=0.05, max_weight=6),
            PlaneType(id=1, max_range=1.0, speed=0.15, max_weight=2),
        ]
        num_airports = 122
        num_initial_cargo = 732
        max_cargo = 852
        soft_deadline_multiplier = 5
        hard_deadline_multiplier = 6
        malfunction_min_duration = 12
        malfunction_max_duration = 48
        malfunction_rate = 0.18
        

        # Decide airport generator
        airport_generator = RandomAirportGenerator(
            max_airports=num_airports,
            make_drop_off_area=True,
            make_pick_up_area=True,
            num_drop_off_airports=2,
            num_pick_up_airports=2,
            mapgen=PlainMapGenerator(),
        )

        # Decide route generator
        route_generator = RouteByDistanceGenerator(
            route_ratio=2,
            poisson_lambda=malfunction_rate,
            malfunction_generator=EventIntervalGenerator(
                                        min_duration=malfunction_min_duration,
                                        max_duration=malfunction_max_duration),
        )

        # Decide cargo generator
        cargo_generator = DynamicCargoGenerator(
            cargo_creation_rate=1/100,
            max_cargo_to_create=max_cargo-num_initial_cargo,
            num_initial_tasks=num_initial_cargo,
            max_weight=3,
            max_stagger_steps=max_cycles/2,
            soft_deadline_multiplier=soft_deadline_multiplier,
            hard_deadline_multiplier=hard_deadline_multiplier,
        )

        # Create the environment
        env = AirliftEnv(
                world_generator=AirliftWorldGenerator(
                plane_types=plane_types,
                airport_generator=airport_generator,
                route_generator=route_generator,
                cargo_generator=cargo_generator,
                airplane_generator=AirplaneGenerator(num_of_agents=num_agents),
                max_cycles=max_cycles
                ),
            )
        
        return env


    def next(self):
        """
        Returns the environment and epsilon for the next episode
        """
        
        # Decide environment type
        env = self.small_env_with_purturbations()

        self.episode += 1
        return env

In [3]:
# plane_reward_vector(), cargo_reward_vector()

def plane_reward_vector(actions, obs):
    """
    Returns the reward vector for the cargo & planes in the observation.
    r_plane = -route_cost

    Args:
        actions (dict, dict): The actions for the timestep - plane_id: {
            'priority': int
            'cargo_to_load': list, int
            'cargo_to_unload': list, int
            'destination': int
        }
        obs (dict): The observation for the timestep
    
    Returns:
        plane_reward_vector (list): The reward for each plane in the observation.
    """

    plane_reward_vector = []
    route_map = obs['a_0']['globalstate']['route_map'][0]
    for plane_id, v in actions.items():
        if v['destination'] != 0:
            plane_reward_vector.append(-route_map.get_edge_data\
                    (obs[plane_id]['current_airport'], v['destination'])['cost'])
        else:
            plane_reward_vector.append(0)
    
    return plane_reward_vector


def cargo_reward_vector(obs, t):
    """
    Returns the reward vector for the cargo & planes in the observation.
    r_cargo = {
        0 if cargo is at the destination
        max(0, t-soft_deadline) / (hard_deadline - soft_deadline) else if t < hard_deadline
        1 else
    }

    Args:
        obs (dict): The observation for the timestep
        t (int): The current timestep
    
    Returns:
        cargo_reward_vector (list): The reward for each cargo in the observation.
        cargo_mapping_vector (list): The mapping of the cargo to the reward vector.
    """

    cargo_reward_vector = []
    cargo_mapping_vector = []
    active_cargo = obs['a_0']['globalstate']['active_cargo']
    on_a_plane = set()

    for _, a in obs.items():
        on_a_plane.update(a['cargo_onboard'])
    
    for c in active_cargo:

        cargo_mapping_vector.append(c.id)

        if (c.id not in on_a_plane) and ((c.destination == c.location) or (c.location == 0)):
            cargo_reward_vector.append(0)
        elif t < c.hard_deadline:
            cargo_reward_vector.append(
                max(0, t-c.soft_deadline) / (c.soft_deadline - c.hard_deadline))
        else:
            cargo_reward_vector.append(1)

    return cargo_reward_vector, cargo_mapping_vector



In [4]:
# heuristic_value()

def heuristic_value(obs, f_obs, t, C_c=1, C_p=1, C_g=5, C_s=1.3, lambda_=0.99, prev_cargo=None):
    """
    Returns the heuristic value for the given state:
    -Sum[c](lambda^(deadline - t) * ((P_c - D_c)^2 + C_g*Grounded_c))
    -Sum[c, p~S](C_p * (P_p - P_c)^2)*grounded

    Attempted to scale like the reward structure for the environment.
    Uses hard_deadline for cargo.
    The airlift documentation says that the scaled flight cost is divided by totalcargo, but
    I don't know what totalcargo is, so I'm using the total weight of the cargo.

    Args:
        obs: The observation
        f_obs: The formatted observation
        t: The current timestep
        C_c: The cargo cost per unit distance
        C_p: The plane cost per unit distance
        C_g: The cost to load a cargo
        C_s: The scaling factor
        lambda_: The discount factor
    
    Returns:
        value
    """

    route_map = obs['a_0']['globalstate']['route_map'][0]
    plane_dict = {}
    for p in f_obs['agents']['agents']:
        cap = p['max_weight'] - p['current_weight']
        while cap>0:
            if cap not in plane_dict:
                plane_dict[cap] = []
            plane_dict[cap].append((p['location'], p['max_weight']))
            cap -= 1


    cargo_cost = 0
    plane_cost = 0
    total_cargo = 0

    for c in obs['a_0']['globalstate']['active_cargo']:

        total_cargo += c.weight

        # Sum[c](lambda^(deadline - t) * ((P_c - D_c)^2 + C_g*Grounded_c))
        if c.location==0:
            if prev_cargo is not None and c.id in prev_cargo:
                x, y = prev_cargo[c.id]
            else:
                # print('Cargo not found')
                # NOTE: This is hopefully a temporary fix, the id seems to not be in the prev_cargo very often
                continue
        else:
            x, y = route_map.nodes[c.location]['pos']
        dest_x, dest_y = route_map.nodes[c.destination]['pos']
        grounded = 0 if c.id in f_obs['cargo']['cargo_on_a_plane'] else 1
        cargo_cost += lambda_**(c.hard_deadline - t) * (((x-dest_x)**2 + (y-dest_y)**2)**(0.5) + C_g*grounded)

        if grounded==1 and c.weight in plane_dict:
            # Sum[c, p~S](C_p * (P_p - P_c)^2)*grounded
            plane = random.choice(plane_dict[c.weight])
            x_plane, y_plane = route_map.nodes[plane[0]]['pos']
            plane_cost += (C_p * ((x_plane-x)**2 + (y_plane-y)**2)**(1/2)) / plane[1]
        else:
            plane = random.choice(f_obs['agents']['agents'])
            plane_cost += C_p * (((x-dest_x)**2 + (y-dest_y)**2)**(0.5) + C_g*grounded) / plane['max_weight']


    return (-cargo_cost - plane_cost/total_cargo)*C_s


def create_prev_cargo(obs, prev_cargo=None):
    prev_cargo = {}
    for c in obs['a_0']['globalstate']['active_cargo']:
        if c.location==0:
            if prev_cargo is not None and c.id in prev_cargo:
                x, y = prev_cargo[c.id]
            else:
                # print('Cargo not found')
                continue
        else:
            x, y = obs['a_0']['globalstate']['route_map'][0].nodes[c.location]['pos']
        prev_cargo[c.id] = (x, y)
    return prev_cargo

In [5]:
# calculate_advantages()

def calculate_advantages(R, gamma=0.99, lam=0.95):
    """
    Calculates the advantages for the given rewards and values.
    Advantage = R_t + gamma**(len(R)-1-t)*V_T - V_t if V_t > V_heuristic
                R_t + gamma**(len(R)-1-t)*V_h_t+1 - V_t if V_h_t <= V_heuristic

    Args:
        R (list, dict): The transition for each timestep {
            't': t,
            'plane_assignments': y['plane_assignments_mtx'],
            'plane_distribution': y['plane_assignments_dist'],
            'cargo_assignments': y['cargo_assignments_mtx'],
            'cargo_distribution': y['cargo_assignments_dist'],
            'plane_reward_vector': plane_reward_vector(actions, obs),
            'agent_map': agent_map,
            'cargo_map': cargo_map,
            'value_estimation': v.forward(x),
            'cargo_reward_vector': cargo_reward_vector(obs, t),
            'cargo_mapping_vector': cargo_mapping_vector,
        }
        v (nn.Module): The state value estimator.
        gamma (float): The discount factor.
        lam (float): The GAE factor.
    
    Returns:
        list: The advantages for each timestep.
    """

    # Calculate Average Reward
    num_cargo = 0
    for r in R:
        if r['cargo_mapping_vector'] != []:
            num_cargo = max(num_cargo, max(r['cargo_mapping_vector']))
    avg_reward_plane = sum([sum(r['plane_reward_vector']) for r in R])/\
        len(R[0]['plane_reward_vector'])
    avg_reward_cargo = sum([sum(r['cargo_reward_vector']) for r in R])/\
        num_cargo
    print("Average Reward Per Plane:", avg_reward_plane)
    print("Average Reward Per Cargo:", avg_reward_cargo)


    # Calculate discounted rewards
    running_cargo_reward = [0]*(num_cargo+1)
    running_plane_reward = [0]*len(R[0]['plane_reward_vector'])
    for t in reversed(range(len(R))):

        # Cargo
        cargo_discounted_reward_vector = []
        for i, rew in enumerate(R[t]['cargo_reward_vector']):
            idx = R[t]['cargo_mapping_vector'][i]
            running_cargo_reward[idx] = rew + gamma*running_cargo_reward[idx]
            cargo_discounted_reward_vector.append(running_cargo_reward[idx])
        R[t]['cargo_discounted_reward_vector'] = torch.tensor(cargo_discounted_reward_vector)

        # Planes
        plane_discounted_reward_vector = []
        for i, rew in enumerate(R[t]['plane_reward_vector']):
            running_plane_reward[i] = rew + gamma*running_plane_reward[i]
            plane_discounted_reward_vector.append(running_plane_reward[i])
        R[t]['plane_discounted_reward_vector'] = torch.tensor(plane_discounted_reward_vector)
    
    # Calculate advantages
    total_adv = 0
    for t in range(len(R)):
        # sum the discounted rewards for the cargo and planes
        cargo_rew_total = R[t]['cargo_discounted_reward_vector'].sum()
        plane_rew_total = R[t]['plane_discounted_reward_vector'].sum()
        rew = cargo_rew_total + plane_rew_total
        value_estimation_t = R[t]['value_estimation']
        value_estimation_T = R[-1]['value_estimation']
        if t < len(R)-1:
            adv = rew + (gamma**(len(R)-1-t))*value_estimation_T - value_estimation_t
        else:
            adv = rew - value_estimation_t
        
        R[t]['advantage'] = adv
        total_adv += adv

    # Normalize the advantages
    mean_adv = total_adv / len(R)
    std_adv = sum([(r['advantage']-mean_adv)**2 for r in R]) / len(R)
    for r in R:
        r['advantage'] = (r['advantage'] - mean_adv) / (std_adv + 1e-8)

    
    return R

        

In [6]:
# PPOLoss

class PPOPolicyLoss(nn.Module):

    def __init__(self):
        super(PPOPolicyLoss, self).__init__()
        self.running_loss = torch.tensor(0, dtype=torch.float32)
        self.num_samples = torch.tensor(0, dtype=torch.float32)
    
    def reset_init(self):
        self.running_loss = torch.tensor(0, dtype=torch.float32)
        self.num_samples = torch.tensor(0, dtype=torch.float32)
    
    def add(self, log_prob_new, log_prob_old, advantage, clip=0.2):
        ratio = torch.exp(log_prob_new - log_prob_old)
        self.running_loss -= torch.squeeze(torch.sum(torch.min(ratio*advantage, torch.clamp(ratio, 1-clip, 1+clip)*advantage)))
        self.num_samples += torch.tensor(1)
    
    def forward(self):
        return self.running_loss / self.num_samples        


class PPOValueLoss(nn.Module):

    def __init__(self):
        super(PPOValueLoss, self).__init__()
        self.running_loss = torch.tensor(0, dtype=torch.float32)
        self.num_samples = torch.tensor(0, dtype=torch.float32)
    
    def reset_init(self):
        self.running_loss = torch.tensor(0, dtype=torch.float32)
        self.num_samples = torch.tensor(0, dtype=torch.float32)
    
    def add(self, plane_rew_pred, cargo_rew_pred, plane_rew_actual, cargo_rew_actual):
        self.running_loss += torch.squeeze(torch.sum(torch.square(plane_rew_pred - plane_rew_actual)))
        self.running_loss += torch.squeeze(torch.sum(torch.square(cargo_rew_pred - cargo_rew_actual)))
        self.num_samples += torch.tensor(1)
    
    def forward(self):
        return self.running_loss / self.num_samples        


In [7]:
# step()

def step(pi, v, minibatch, train_model, optimizer_pi, optimizer_v, loss_pi, loss_v, clip=0.2):
    """
    Updates the policy and value function parameters based on the minibatch.
    Updates the policy, alternating between backpropogating from the plane assignments
        head and the cargo assignments head. Then updates the value function.
    Optimizes:
        - L_pi = 
    
    Args:
        pi (nn.Module): The policy to adjust parameters for.
        v (nn.Module): The state value estimator.
        minibatch (list, dict): The minibatch of transitions {
            't': t,
            'obs': obs,
            'plane_assignments': y['plane_assignments_mtx'],
            'plane_distribution': y['plane_assignments_dist'],
            'cargo_assignments': y['cargo_assignments_mtx'],
            'cargo_distribution': y['cargo_assignments_dist'],
            'plane_reward_vector': plane_reward_vector(actions, obs),
            'agent_map': agent_map,
            'cargo_map': cargo_map,
            'value_estimation': v.forward(x),
            'cargo_reward_vector': cargo_reward_vector(obs, t),
            'cargo_mapping_vector': cargo_mapping_vector,
            'cargo_discounted_reward_vector': cargo_discounted_reward_vector,
            'plane_discounted_reward_vector': plane_discounted_reward_vector,
            'advantage': advantage,
        }
        train_model (int {0, 1}): Which head to train. 0=plane, 1=cargo

    Returns:
        pi (nn.Module): The updated policy.
        v (nn.Module): The updated value function.
    """

    pi.requires_grad_(True)
    v.requires_grad_(True)
    loss_pi.reset_init()
    loss_v.reset_init()

    for sample in minibatch:

        # If there are no cargo or planes, skip the sample
        if sample['cargo_mapping_vector'] == [] or sample['plane_reward_vector'] == []:
            print("No cargo or planes")
            continue

        # Calculate the loss for the policy
        r = pi(sample['x'])
        if train_model == 0:
            plane_assignments = torch.argmax(sample['plane_assignments'], dim=1)
            log_prob_new = Categorical(r['plane_assignments_dist']).log_prob(plane_assignments)
            log_prob_old = Categorical(sample['plane_distribution']).log_prob(plane_assignments)
        else:
            cargo_assignments = torch.argmax(sample['cargo_assignments'], dim=1)
            log_prob_new = Categorical(r['cargo_assignments_dist']).log_prob(cargo_assignments)
            log_prob_old = Categorical(sample['cargo_distribution']).log_prob(cargo_assignments)
        loss_pi.add(log_prob_new, log_prob_old, sample['advantage'], clip=clip)

        # Calculate the loss for the value function
        r = v(sample['x'])
        loss_v.add(r['plane_reward_vector'], r['cargo_reward_vector'],
                   sample['plane_discounted_reward_vector'], sample['cargo_discounted_reward_vector'])

    L_clip = loss_pi()
    L_v = loss_v()

    # Backpropogate and optimize the policy
    optimizer_pi.zero_grad()
    L_clip.backward(retain_graph=True)
    optimizer_pi.step()

    # Backpropogate and optimize the value function
    optimizer_v.zero_grad()
    L_v.backward()
    optimizer_v.step()

    return pi, v

In [8]:
# average_edge_cost()

def average_edge_cost(obs):
    """
    Returns the average edge cost for the given observation per unit distance.
    """

    route_map = obs['a_0']['globalstate']['route_map'][0]
    total_cost = 0
    total_distance = 0
    for u, v, data in route_map.edges(data=True):
        total_cost += data['cost']
        x1, y1 = route_map.nodes[u]['pos']
        x2, y2 = route_map.nodes[v]['pos']
        total_distance += ((x1-x2)**2 + (y1-y2)**2)**(1/2)
    
    return total_cost / total_distance

In [29]:
# PPO()

def PPO(pi, v, max_cycles, seed, num_episodes, path, scheduler, loss_pi,
        loss_v, num_updates, optimizer_pi, optimizer_v, batch_size=32):
    """
    Pytorch implimentation of PPO
    """

    random.seed(seed)
    train_model = 0

    for episode in range(num_episodes):
        print('\nStarting Episode', episode)

        # Gather pi(s, a, g) & rewards
        env = scheduler.next()
        obs = env.reset(seed=seed)
        avg_edge_cost = average_edge_cost(obs) * nx.diameter(obs['a_0']['globalstate']['route_map'][0])
        R = []

        # Run Episode
        with torch.no_grad():
            for t in tqdm(range(max_cycles)):

                # Execute Timestep
                x = format_obs(obs, t)
                if x is None:
                    continue
                y = pi.forward(x)

                # Create a map of agent and cargo ids
                agent_map = []
                for agent in x['agents']['agents']:
                    agent_map.append(agent['id'])
                agent_map.append(None)

                cargo_map = []
                for cargo in x['cargo']['cargo']:
                    cargo_map.append(cargo['id'])

                # Get actions
                actions = post_process(x=x, y=y, agent_map=agent_map, cargo_map=cargo_map)

                # Create transition (s, a, r) .. don't need the next state for PPO
                transition = {
                    't': t,
                    'x': x,
                    'plane_assignments': y['plane_assignments_mtx'],
                    'plane_distribution': y['plane_assignments_dist'],
                    'cargo_assignments': y['cargo_assignments_mtx'],
                    'cargo_distribution': y['cargo_assignments_dist'],
                    'plane_reward_vector': plane_reward_vector(actions, obs),
                    'agent_map': agent_map,
                    'cargo_map': cargo_map,
                    'value_estimation': v.forward(x)['total_reward'].detach(),
                    #'heuristic_value': heuristic_value(obs, x, t, C_p=avg_edge_cost, C_g=5, C_s=1.3, lambda_=0.99, prev_cargo=prev_cargo),
                }
                

                # Step environment
                obs, rews, dones, infos = env.step(actions=actions)
                for k, val in rews.items():
                    if val<-0.0005032:
                        print(k, val)

                # Add rewards to transition & transition to replay buffer
                transition['cargo_reward_vector'], transition['cargo_mapping_vector'] =\
                    cargo_reward_vector(obs, t+1)

                # # Print warnings
                # if infos is not None:
                #     for _, w in infos.items():
                #         if w['warnings'] != []:
                #             print(w['warnings'])
                
                # If done, then break
                done = True
                for _, d in dones.items():
                    if not d:
                        done = False
                        break
                if done:
                    break

                R.append(transition)
            
            # Calculate advantage estimates
            print('Calculating Advantages')
            R = calculate_advantages(R)
                
        # Update weights
        print('Updating Weights')
        
        random.shuffle(R)
        for epoch in range(num_updates):
            if (epoch+1)*batch_size > len(R):
                minibatch = R[-batch_size:-1]
                random.shuffle(R)
            else:
                minibatch = R[epoch*batch_size:(epoch+1)*batch_size]
            # NOTE: pi and v gradient tracking is off at this point
            pi, v = step(pi=pi, v=v, minibatch=minibatch, train_model=train_model, optimizer_pi=optimizer_pi,
                         optimizer_v=optimizer_v, loss_v=loss_v, loss_pi=loss_pi, clip=0.2)
            train_model = 1-train_model
        
        # Save the models
        save_path = path + '_episode_' + str(episode)
        torch.save(pi.state_dict(), save_path + '_pi.pth')
        torch.save(v.state_dict(), save_path + '_v.pth')
        print('Saved models to', path)

In [30]:
# # PPO()

# class PPO:


#     def __init__(self, seed):

#         random.seed(seed)
#         self.replay_buffer = []


#     def collect_samples(timesteps_per_iteration=2000):

#         # reset replay buffer

#         while True:

#             # if we need a new env, get one

#             # run timestep

#             # store transition

#             timesteps_per_iteration -= 1
#             if timesteps_per_iteration==0:
#                 return


#     def __call__(self):
        
#         # Collect timesteps_per_iteration samples

# TODO:
- Make the environment scheduler more random

Notes:

- select batch_size prioritized samples starting at the successful delivery of a cargo and work backwards..something at the edge of a threshold of value
- replace heuristic value fucntion with a neural net trained on samples from a shortest path solution

- Maybe we should mix up episodes so the model isn't trained on one graph for an entire epoch?
        like, make multiple episodes per iteration, and shuffle all samples
- Maybe train on multiple episodes at once to reduce a bias towards a single map

Try different schedules to train it:
- all environments difficult
- start with small environments and get bigger and more disruptive

In [31]:
# Setup

# Parameters
max_cycles = 5000
seed = 2025
num_episodes = 1000
version = 'v2'
path = 'solution/models/'+version+'/'+version
num_updates = 10
scheduler = Scheduler(seed=seed, num_episodes=num_episodes)
learn_rate = 0.0001
batch_size=1024

# Initialize solution & critic
policy_path = 'solution/models/'+version+'/policy_'+version+'_base.pth'
value_path = 'solution/models/'+version+'/value_'+version+'_base.pth'
pi = Policy(get_config('v1')['policy'])
v = Value(get_config('v1')['value'])
v.load_state_dict(torch.load(value_path))
pi.load_state_dict(torch.load(policy_path))

# Initialize optimizers & loss fns
optimizer_pi = torch.optim.Adam(pi.parameters(), lr=learn_rate)
optimizer_v = torch.optim.Adam(v.parameters(), lr=learn_rate)
loss_pi = PPOPolicyLoss()
loss_v = PPOValueLoss()

In [32]:
# Run PPO
# NOTE: This current run will show if the heuristic helps it converge faster, but it is not the final solution since
# there's still something wrong with the way we handle cargo edge cases. Test Failed.
# NOTE: Test of just PPO with no heuristic. Failed. Needs refactoring or rewards shaping
PPO(pi=pi, v=v, max_cycles=max_cycles, seed=seed, num_episodes=num_episodes,
    path=path, scheduler=scheduler, num_updates=num_updates, loss_pi=loss_pi,
    loss_v=loss_v, optimizer_pi=optimizer_pi, optimizer_v=optimizer_v, batch_size=batch_size)


Starting Episode 0


  8%|▊         | 384/5000 [00:07<01:21, 56.93it/s]

a_0 -1.0004891
a_1 -1.0004904
a_0 -1
a_1 -1.0004904
a_0 -1
a_1 -1.0004904
a_0 -1
a_1 -1.0004904
a_0 -1
a_1 -1
a_0 -1
a_1 -1
a_0 -1
a_1 -1
a_0 -1
a_1 -1
a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2
a_1 -2


  8%|▊         | 396/5000 [00:07<01:26, 52.95it/s]

a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2
a_1 -2
a_0 -2.0004891000000002
a_1 -2
a_0 -2.0004891000000002
a_1 -2
a_0 -2.0004891000000002
a_1 -2.0004891000000002
a_0 -2.0004891000000002
a_1 -2.0004891000000002


...

 38%|███▊      | 1912/5000 [00:37<01:20, 38.47it/s]

a_0 -4.0004904
a_1 -4.0004904
a_0 -4.0004904
a_1 -4.0004904
a_0 -4.0004904
a_1 -4.0004904
a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4
a_0 -4
a_1 -4
a_0 -4
a_1 -4
a_0 -4
a_1 -4.0004904


 38%|███▊      | 1920/5000 [00:37<01:00, 50.54it/s]


a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4.0004904
a_0 -4
a_1 -4.0004904


KeyboardInterrupt: 